# Case - Modelo Preditivo para aprovacao de credito

## 02 - Submeter os dados aos modelos de Machine Learning
<p>
Modelos de score de crédito calculam a probabilidade de inadimplência e são uma das
principais ferramentas utilizadas por diversas empresas para aprovar ou negar um crédito.
O objetivo deste desafio é criar um modelo preditivo calculando a probabilidade de
inadimplência de cada novo pedido de crédito.
    
    
<b>Dados</b>:
Cada linha representa um cliente e as colunas representam os dados (informações) desses
clientes.
A variável resposta é a coluna inadimplente, que indica se o tomador veio a se tornar
inadimplente(1) ou não(0).
    
    
<b>As variáveis da base de dados são descritas abaixo:</b>
    <ul>
        <li>idade: A idade do cliente</li>
        <li>numero_de_dependentes: O número de pessoas dependentes do cliente.</li>
        <li>salario_mensal: Salário mensal do cliente.</li>
        <li>numero_emprestimos_imobiliarios: Quantidade de empréstimos imobiliários que o
cliente possui em aberto.</li>
        <li>util_linhas_inseguras: Quanto que o cliente está usando, relativamente ao limite
dele, de linhas de crédito que não são seguradas por qualquer bem do tomador
e.g: imoveis, carros etc.</li>
        <li>vezes_passou_de_30_59_dias: Número de vezes que o cliente atrasou, entre 30
e 59 dias, o pagamento de um empréstimo.</li>
        <li>razao_debito: Razão entre as dívidas e o patrimônio do tomador. razão débito =
Dividas/Patrimônio</li>
        <li>numero_linhas_crdto_aberto: Número de empréstimos em aberto pelo cliente.</li>
        <li>numero_de_vezes_que_passou_60_89_dias: Número de vezes que o cliente
atrasou, entre 60 e 89 dias, o pagamento de um empréstimo.</li>        
    </ul>
    
Obs: Estes dados foram retirados de terceiros, portanto é possível que existam
incoerências, o que é perfeitamente comum em dados reais.


<b>Objetivo:</b>
Construir qualquer modelo preditivo utilizando o arquivo treino.csv.
Utilize este modelo para gerar as previsões na base teste.csv, inserindo uma nova coluna
na tabela de dados do arquivo teste.csv que contenha as previsões e nomeie esta coluna
com o nome "inadimplente".
Espera-se que não seja necessário mais que um dia para a resolução deste problema.

</p>

<p>
    <b>Author: </b>Joao Paulo Ribeiro dos Santos<br>
    <b>E-mail: </b>joao1615@gmail.com  <b>ou</b>  joao1615@faculdadefia.edu.br<br>
    <b>Telefone: </b> (11)9 6816-5227
</p>

In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [30]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Joao Paulo" --iversions

Author: Joao Paulo

seaborn   : 0.11.1
pandas    : 1.2.1
numpy     : 1.19.2
matplotlib: 3.3.2



In [34]:
# Importacao dos datasets
df_train = pd.read_csv('processed_data/train_treated_v2.csv')
df_test = pd.read_csv('processed_data/test_treated_v2.csv')

In [35]:
# Tipo dos dados
df_train.dtypes

util_linhas_inseguras                    float64
idade                                      int64
vezes_passou_de_30_59_dias                 int64
razao_debito                             float64
salario_mensal                           float64
numero_linhas_crdto_aberto                 int64
numero_vezes_passou_90_dias                int64
numero_emprestimos_imobiliarios            int64
numero_de_vezes_que_passou_60_89_dias      int64
numero_de_dependentes                    float64
inadimplente                               int64
possui_mais_dividas                        int64
salario_mensal_log2                      float64
dtype: object

In [36]:
# Tipo dos dados
df_test.dtypes

util_linhas_inseguras                    float64
idade                                      int64
vezes_passou_de_30_59_dias                 int64
razao_debito                             float64
salario_mensal                           float64
numero_linhas_crdto_aberto                 int64
numero_vezes_passou_90_dias                int64
numero_emprestimos_imobiliarios            int64
numero_de_vezes_que_passou_60_89_dias      int64
numero_de_dependentes                    float64
possui_mais_dividas                        int64
salario_mensal_log2                      float64
dtype: object